In [6]:
#@title setup packages

# Install necessary packages
!pip install langchain_chroma
!pip install langchain_openai
!pip install langchain
!pip install langchain_community

# Import necessary modules
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
import bs4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00


# Loading Dataset

In [7]:
# Loading and storing datasets
# Load documents (e.g., from local storage)
#document_loader = DocumentLoader("path_to_local_storage")

#Loading from web
# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

43131

# Splitting dataset

In [10]:
#Define different text splitter methods
# Recursive Character Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
recursive_splitter = RecursiveCharacterTextSplitter(  chunk_size=1000, chunk_overlap=200, add_start_index=True)
all_splits = recursive_splitter.split_documents(docs)
len(all_splits)

# HTML Header Text Splitter
#html_splitter = HTMLHeaderTextSplitter()

# Markdown Header Text Splitter
#markdown_splitter = MarkdownHeaderTextSplitter()

# Code Text Splitter
#code_splitter = CodeTextSplitter()

# Token Text Splitter
#token_splitter = TokenTextSplitter()

# Character Text Splitter
#character_splitter = CharacterTextSplitter()

# Semantic Chunker
#semantic_chunker = SemanticChunker()

# AI21 Semantic Text Splitter
#ai21_splitter = AI21SemanticTextSplitter()

66

In [ ]:

# Define documents
all_splits = document_loader.load_documents()

# Embed and store document splits
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# Embedding documents
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# Other embedding models are available in:
# - lang_chain_embedding_models
# - sentence-transformers
# - Kaggle (includes all ML models, not just embedding ones)

# Retrieving top k relevant embeddings
def retrieve_top_k(vectorstore, query, k=5, method='cosine'):
    if method == 'cosine':
        return vectorstore.retrieve(query, k=k, method='cosine')
    elif method == 'dot_product':
        return vectorstore.retrieve(query, k=k, method='dot_product')
    elif method == 'euclidean':
        return vectorstore.retrieve(query, k=k, method='euclidean')
    else:
        raise ValueError("Unsupported retrieval method")

# Generate final answers
from langchain import LLM
# Choose an LLM model
llm = LLM()

# Define RAG Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | prompt
   | llm
   | StrOutputParser()
)

# Example usage of the RAG chain
for chunk in rag_chain.stream("What is Task Decomposition?"):
   print(chunk, end="", flush=True)

# The prompt is expected to be a dict with keys "context" and "question".
# retriever | format_docs passes the question through the retriever, generating Document objects, and then to format_docs to generate strings;
# RunnablePassthrough() passes through the input question unchanged;
# llm runs the inference;
# StrOutputParser() plucks the string content out of the LLM's output message.

# Example usage of the retriever
query = "What is Task Decomposition?"
top_k_docs = retrieve_top_k(vectorstore, query)

for doc in top_k_docs:
    print(doc.page_content)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 

ImportError: cannot import name 'DocumentLoader' from 'langchain' (/usr/local/lib/python3.10/dist-packages/langchain/__init__.py)